In [ ]:
# Geometry
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union

# Data handling and combinations
from itertools import cycle, combinations
from collections import defaultdict

# Plotly imports
import plotly.graph_objects as go
from plotly.colors import sequential

# Matplotlib for additional plotting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Standard Library Imports
import time
import colorsys

# Scientific Computing
import numpy as np
import sympy as sp
from sympy import symbols, diff
from scipy.spatial import ConvexHull

# PyCalphad (Thermodynamics Calculations & Plotting)
from pycalphad import Database, calculate, equilibrium, variables as v
from pycalphad.plot.utils import phase_legend
from pycalphad import ternplot

# Color Processing
from skimage.color import deltaE_ciede2000, rgb2lab

In [ ]:
def projection_function(f1, f2):
    '''
    Computes the projection of f2 onto f1.
    
    Parameters:
        f1 : sympy expression

        f2 : sympy expression
    
    Returns:
        proj : sympy expression

        variables : tuple

        pvariables : tuple
    '''
    free_symbols_f1 = f1.free_symbols
    free_symbols_f2 = f2.free_symbols

    variables = tuple(free_symbols_f1.intersection(free_symbols_f2))
    pvariables = tuple(symbols(f'{symbol}_p') for symbol in variables)

    sum_terms = [(variable - pvariables[i])*diff(f1, variable).subs(variable, pvariables[i]) for i, variable in enumerate(variables)]
    proj = f2 - f1.subs(dict(zip(variables, pvariables))) - sum(sum_terms)

    return proj.expand(), variables, pvariables

In [ ]:
def recursive_discriminant(expr, vars):
    '''
    Takes the discriminant over all variables in a given expression.

    Parameters:
        expr (sympy expression): The expression to take the discriminant of.

    Returns:
        discriminant (sympy expression): The discriminant of the expression.
    '''
    vars_list = list(vars)
    def recursive_discriminant_helper(expr, vars):
        # if there are no variables, return the expression
        if vars == []:
            return expr
        
        for var in vars_list:
            if expr.has(var):
                vars_list.remove(var)
                return sp.discriminant(recursive_discriminant(expr, vars_list), var)
            
    return recursive_discriminant_helper(expr, vars_list)

In [ ]:
x, y = sp.symbols('x y')

# define your funcitons here in explicit form
f1 = 2*x**2 + y**2
f2 = (x+3)**2 + (y+2)**2 + 1

functions = [sp.lambdify((x, y), f1, 'numpy'), sp.lambdify((x, y), f2, 'numpy')]

In [ ]:
fig = go.Figure()

# Define the range for x and y and create a mesh
x_vals = np.linspace(-8, 7, 300)
y_vals = np.linspace(-8, 7, 300)
x_mesh, y_mesh = np.meshgrid(x_vals, y_vals)

# Define the color scales
colorscales = [
    'Viridis',
    'emrld',
    'Inferno',
    'Cividis',
    'Turbo',
    'Bluered'
]

# Plot all the energy surfaces
for i, f in enumerate(functions):
    z_vals = f(x_mesh, y_mesh)
    fig.add_trace(go.Surface(
        z=z_vals, x=x_mesh, y=y_mesh,
        colorscale=colorscales[i],
        showscale=False
    ))

fig.update_layout(showlegend=False,
                width=600,
                height=600, 
                scene=dict(
                    xaxis_title='X',
                    yaxis_title='Y',
                    zaxis_title='Z',
                    zaxis=dict(range=[-1, 20])),  # Adjust the range as needed
                    coloraxis_showscale=False,
                title='3D Phases with 2D Coexistance')

fig.show()

In [ ]:
disc_funcs = []
for pair in [(f1, f2), (f2, f1)]:
    proj, vars, pvars = projection_function(pair[0], pair[1])
    disc = recursive_discriminant(proj, vars)
    disc_func = sp.lambdify(pvars, disc, 'numpy')
    disc_funcs.append(disc_func)

for func in disc_funcs:
    Z = func(x_mesh, y_mesh)
    plt.contour(x_mesh, y_mesh, Z, levels=[0])
plt.show()

In [ ]:
fig = go.Figure()

# Define the range for x and y and create a mesh
x_vals = np.linspace(-8, 7, 300)
y_vals = np.linspace(-8, 7, 300)
x_mesh, y_mesh = np.meshgrid(x_vals, y_vals)

# Define the color scales
colorscales = [
    'Viridis',
    'emrld',
    'Inferno',
    'Cividis',
    'Turbo',
    'Bluered'
]

colors = [
    'blue',
    'green',
    'red'
]

# Plot all the energy surfaces
for i, f in enumerate(functions):
    z_vals = f(x_mesh, y_mesh)
    fig.add_trace(go.Surface(
        x=x_mesh, y=y_mesh, z=z_vals, 
        colorscale=colorscales[i],
        showscale=False
    ))


for i, disc_func in enumerate(disc_funcs):
    # Create contour plot
    Z = disc_func(x_mesh, y_mesh)
    contour_set = plt.contour(x_mesh, y_mesh, Z, levels=0)
    plt.close()

    all_segments = []
    for seg_level in contour_set.allsegs:
        # seg_level is a list of arrays; each array is shape (N, 2)
        for seg in seg_level:
            all_segments.append(seg)

    data = all_segments[1]
            
    fig.add_trace(
        go.Scatter3d(
            x=data[:, 0],
            y=data[:, 1],
            z = np.zeros_like(data[:, 0]),
            mode="lines",
            line=dict(color=colors[i]),
        )
    )


fig.update_layout(showlegend=False,
                width=600,
                height=600, 
                scene=dict(
                    xaxis_title='X',
                    yaxis_title='Y',
                    zaxis_title='Z',
                    zaxis=dict(range=[-1, 20])),  # Adjust the range as needed
                    coloraxis_showscale=False,
                title='3D Phases with 2D Coexistance')

fig.show()